In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd
from scipy.stats import norm

import sys
sys.path.append("../")
from bargains_linear import * 

In [77]:
def solve_lamb(df):
    p1 =  df['p1'].mean()
    phi1 = df['phi1'].mean()
    phi2 = df['phi2'].mean()
    s1 = df['s1'].mean()
    s2 = df['s2'].mean()
    lambda_val = ( df['p1'] - (2*df['phi1'] + df['phi2'])/3 ).mean()
    
    return lambda_val


def calc_c2(phi1,phi2,beta2,lambda_val):
    return (3.0*beta2*lambda_val - 3.0*lambda_val - 2.0*phi1 + 2.0*phi2)/(beta2 - 1.0)


def solve_beta_helper(phi1,phi2,lambda_val,c2):
    return (3.0*lambda_val + 2.0*phi1 - 2.0*phi2 - c2)/(3.0*lambda_val - c2)


def solve_beta(beta2_new,c2,c2_avg):
    w = norm.pdf(c2,scale=1,loc=0)
    w= w/(w.mean())
    #print(w.min(),w.max(),w.argmin(),w.argmax())
    #print(beta2_new.mean(),(w*beta2_new).mean())
    return (w*beta2_new).mean()


def solve_beta_c2(df):
    phi1 = df['phi1']
    phi2 = df['phi2']
    lambda_val = solve_lamb(df)
    c2_avg = 0 #np.random.normal(size=phi1.shape)
    c2 = np.zeros( phi1.shape)
    beta2 = .6
    for i in range(1000):
        beta2_new = solve_beta_helper(phi1,phi2,lambda_val,c2)
        c2_new = calc_c2(phi1,phi2,beta2,lambda_val)

        #update the loop variables
        beta2 = solve_beta(beta2_new,c2_new,c2.mean())
        c2 = calc_c2(phi1,phi2,beta2_new,lambda_val)
    return beta2
    
    


passive_result1 = pd.read_csv('../GMM_est/fake_data/passive_data_3.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data/active_data_3.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data/seq_data_3.csv',index_col=0)
for df in [seq_result1,passive_result1,active_result1]:
    print(solve_beta_c2(df))
    print('----')

0.0944862429066252
----
0.2671338356518551
----
0.3423644953698067
----
